In [ ]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, silhouette_samples, calinski_harabasz_score, davies_bouldin_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Cargar los datos
ruta_al_archivo = 'datos_telecom_preprocesado_1.csv'  # Modificar según la ubicación del archivo
data = pd.read_csv(ruta_al_archivo)

# 2. Preprocesamiento de datos
# Rellenar valores faltantes
data.fillna(data.median(numeric_only=True), inplace=True)
data.fillna('Desconocido', inplace=True)

# Codificar variables categóricas
label_encoders = {}
for col in data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Escalar datos numéricos
scaler = StandardScaler()
data_scaled = pd.DataFrame(scaler.fit_transform(data.select_dtypes(include=['number'])), 
                           columns=data.select_dtypes(include=['number']).columns)

# 3. Selección de características
X = data_scaled

# 4. Determinación del número óptimo de clusters
wcss = []
silhouette_scores = []

for i in range(2, 11):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(X, kmeans.labels_))

# Método del codo
plt.figure(figsize=(10, 5))
plt.plot(range(2, 11), wcss, marker='o')
plt.title('Método del Codo')
plt.xlabel('Número de Clusters')
plt.ylabel('WCSS')
plt.show()

# Coeficiente de silueta
plt.figure(figsize=(10, 5))
plt.plot(range(2, 11), silhouette_scores, marker='o')
plt.title('Coeficiente de Silueta')
plt.xlabel('Número de Clusters')
plt.ylabel('Puntuación de Silueta')
plt.show()

# Seleccionar el número óptimo de clusters (ajustar según los gráficos)
n_clusters_optimo = 6

# 5. Entrenamiento del modelo K-Means
kmeans = KMeans(n_clusters=n_clusters_optimo, random_state=42)
data['Cluster'] = kmeans.fit_predict(X)

# 6. Validación del modelo
# Coeficiente de Silueta
silhouette_avg = silhouette_score(X, data['Cluster'])
print(f'Coeficiente de Silueta Promedio: {silhouette_avg:.2f}')

# Índice Calinski-Harabasz
calinski_score = calinski_harabasz_score(X, data['Cluster'])
print(f'Índice de Calinski-Harabasz: {calinski_score:.2f}')

# Índice Davies-Bouldin
davies_bouldin = davies_bouldin_score(X, data['Cluster'])
print(f'Índice Davies-Bouldin: {davies_bouldin:.2f}')

# Distribución de clusters
cluster_counts = data['Cluster'].value_counts()
plt.figure(figsize=(8, 5))
sns.barplot(x=cluster_counts.index, y=cluster_counts.values)
plt.title('Distribución de Puntos por Cluster')
plt.xlabel('Cluster')
plt.ylabel('Número de Puntos')
plt.show()

# 7. Visualización con PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)
data['PCA1'], data['PCA2'] = X_pca[:, 0], X_pca[:, 1]

plt.figure(figsize=(10, 8))
sns.scatterplot(data=data, x='PCA1', y='PCA2', hue='Cluster', palette='Set1', alpha=0.7)
plt.title('Visualización de Clusters (PCA)')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend(title='Cluster')
plt.show()

# 8. Gráfico de Siluetas
def plot_silhouette(X, labels, n_clusters):
    silhouette_vals = silhouette_samples(X, labels)
    y_lower, y_upper = 0, 0
    yticks = []

    plt.figure(figsize=(10, 7))
    for i in range(n_clusters):
        cluster_silhouette_vals = silhouette_vals[labels == i]
        cluster_silhouette_vals.sort()
        y_upper += len(cluster_silhouette_vals)
        plt.barh(range(y_lower, y_upper), cluster_silhouette_vals, edgecolor='none', height=1)
        yticks.append((y_lower + y_upper) / 2)
        y_lower += len(cluster_silhouette_vals)

    plt.axvline(silhouette_avg, color="red", linestyle="--")
    plt.yticks(yticks, [f'Cluster {i}' for i in range(n_clusters)])
    plt.ylabel("Clusters")
    plt.xlabel("Coeficiente de Silueta")
    plt.title("Gráfico de Silueta")
    plt.show()

# Graficar siluetas
plot_silhouette(X, data['Cluster'].values, n_clusters_optimo)

# 9. Guardar resultados
data.to_csv('clientes_segmentados_validado.csv', index=False)
print("Los datos segmentados y validados se han guardado en 'clientes_segmentados_validado.csv'.")